# Natural Convection with SpatialPy

## Definition of the model

### Imports and definitions

In [1]:
import os
import sys
sys.path.insert(1, "../..")
import numpy as np
import matplotlib.pyplot as plt

import spatialpy

In [2]:
class Cylinder(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return x[0]**2 + x[1]**2 < 0.1**2

In [3]:
class Wall(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] < -0.5 or x[0] > 0.5: return True
        if x[1] < -0.5 or x[1] > 0.5: return True
        return False

### Model

In [4]:
class NaturalConvection(spatialpy.Model):
    def __init__(self, model_name="Natural Convection"):
        spatialpy.Model.__init__(self, model_name)
        
        # System constants
        L = 1                # characteristic lenght of the cavity (= width = height)
        nxF, nyF = 200, 200  # number of fluid particles in x and y-direction
        Sc = 0.7             # Schmidt number
        Ra = 1e4             # mass transfer Rayleigh (1e4, 1e5, 1e6)
        nW = 3              # number of wall points
        nu = 0.01           # fluid viscosity
        rho0 = 1             # reference fluid density
        c0 = 1              # reference speed of sound (typically, 10* max velocity)
        g = 10
        beta = 1e-6
        P0  = rho0*(g*beta*L)**2    # reference pressure
        
        # Variables
        C = spatialpy.Species(name="C", diffusion_coefficient=(1 / np.sqrt(Sc * Ra)))
        self.add_species(C)
        
        # Discretization
        nxTot = nxF + 2*nW # total number of particles in x-direction (including walls)
        nyTot = nyF + 2*nW # total number of particles in y-direction (including walls)
        
        # Compute domain bounds (including the boundary)
        dx, dy = L/nxF, L/nyF
        xLim = (-0.5-(nW-1)*dx, 0.5+(nW-1)*dx)
        yLim = (-0.5-(nW-1)*dy, 0.5+(nW-1)*dy)
        
        # Compute volume and mass per particle
        vol = (xLim[1]-xLim[0])*(yLim[1]-yLim[0])*1.0 # in 2D simulations, consider z-lenght = 1
        mPP = rho0*vol/(nxTot*nyTot)                   # density * total volume / total number of particles
        
        # Domain
        self.domain = spatialpy.Domain.create_2D_domain(
            xlim=xLim, ylim=yLim, nx=nxTot, ny=nyTot,
            type_id=1, mass=mPP, nu=nu, rho0=rho0, c0=c0, P0=P0, fixed=False
        )
        
        # Types
        self.set_type(Wall(), 2, fixed=True)
        self.set_type(Cylinder(), 3, fixed=True)
        
        # Boundary Conditions
        self.add_boundary_condition(spatialpy.BoundaryCondition(
            type_id=3,
            species="C",
            value=1
        ))
        self.add_boundary_condition(spatialpy.BoundaryCondition(
            type_id=3,
            species="C",
            value=0
        ))
        
        self.timespan(np.arange(0, 1+1e-4, 0.01), timestep_size=1e-4)

In [5]:
model = NaturalConvection()

In [6]:
model.run(debug_level=1)

Compiling Solver.  Build dir: /var/folders/nf/3stvfv_d2m16_v3p14l17rmm0000gn/T/spatialpy_build_sp_uy2ki
make -f /Users/owner/Documents/Undergrade_Research/Dr_Drawert/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/external/ANN/src/Makefile.spatialpy ROOTINC=/Users/owner/Documents/Undergrade_Research/Dr_Drawert/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine
make[1]: Nothing to be done for `default'.
g++ -c   -o main.o /var/folders/nf/3stvfv_d2m16_v3p14l17rmm0000gn/T/spatialpy_build_sp_uy2ki/NaturalConvection_generated_model.c -I/Users/owner/Documents/Undergrade_Research/Dr_Drawert/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/include/ -I/Users/owner/Documents/Undergrade_Research/Dr_Drawert/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/external/ -I/Users/owner/Documents/Undergrade_Research/Dr_Drawert/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/external/ANN/include/ -c -std=c++14 -Wall -O3
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is depreca

SimulationError: Compilation of solver failed, return_code=2